In [ ]:
from google.colab import auth
from datetime import datetime
import os
auth.authenticate_user()
!git clone https://github.com/AIStream-Peelout/flow-forecast.git -b frozen_lay

Cloning into 'flow-forecast'...
remote: Enumerating objects: 16787, done.
remote: Counting objects: 100% (3644/3644), done.
remote: Compressing objects: 100% (1378/1378), done.
remote: Total 16787 (delta 2561), reused 3240 (delta 2245), pack-reused 13143
Receiving objects: 100% (16787/16787), 4.49 MiB | 15.79 MiB/s, done.
Resolving deltas: 100% (12112/12112), done.


In [ ]:
import os
os.chdir('flow-forecast')
!pip install shortuuid==1.0.1
!pip install -r  requirements.txt
!python setup.py develop
!mkdir data

In [ ]:
!gsutil cp gs://coronaviruspublicdata/experiments/10_November_202102_15PM_model.pth .
!gsutil cp gs://coronaviruspublicdata/experiments/10_November_202102_15PM.json .

Copying gs://coronaviruspublicdata/experiments/10_November_202102_15PM_model.pth...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/10_November_202102_15PM.json...
/ [1 files][  2.1 KiB/  2.1 KiB]                                                
Operation completed over 1 objects/2.1 KiB.                                      


In [ ]:
from flood_forecast.deployment.inference import InferenceMode

In [ ]:
!gsutil cp gs://aistream-datasets/flowdb/09405500AZC_flow.csv .

Copying gs://aistream-datasets/flowdb/09405500AZC_flow.csv...
/ [1 files][ 19.6 MiB/ 19.6 MiB]                                                
Operation completed over 1 objects/19.6 MiB.                                     


In [ ]:
import json
with open("/content/flow-forecast/10_November_202102_15PM.json") as f1:
  data = json.load(f1)
# a = InferenceMode(5, 20, data, "09405500AZC_flow.csv", "/content/flow-forecast/09_November_202105_59PM_model.pth",

In [ ]:
from flood_forecast.time_model import PyTorchForecast

In [ ]:
data["weight_path"] = "/content/flow-forecast/09_November_202105_59PM_model.pth"
data["GCS"] = False

data["weight_path"] = "/content/flow-forecast/10_November_202102_15PM_model.pth"
data["weight_path_add"] = {
        "excluded_layers": ["output_dim_layer.weight", "output_dim_layer.bias", "out_length_lay.weight", "out_length_lay.bias" ],
        "frozen_layers": ["dense_shape", "pe", "transformer_enc", "squashed", "unsquashed"]
    },

a = PyTorchForecast("CustomTransformerDecoder", "09405500AZC_flow.csv", "09405500AZC_flow.csv", "09405500AZC_flow.csv", data)

In [ ]:
import torch
a.model.eval()
a.model(torch.rand(2, 10, 4))

In [ ]:
with open("saved_param.json", "w+") as y:
  json.dump(data, y)

In [115]:
from flood_forecast.trainer import train_function
import wandb
import pandas as pd
wandb.init()
data["inference_params"] = {
    "test_csv_path": "/content/flow-forecast/09405500AZC_flow.csv",
    "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
    "datetime_start":"2020-05-09",
          "hours_to_forecast":15, 
          "num_prediction_samples": 100,
}
data["dataset_params"]["class"] = "default"
data["epochs"] = 20
data["dataset_params"]["forecast_length"] = 10
data["dataset_params"]["training_path"] = "/content/flow-forecast/09405500AZC_flow.csv"
data["dataset_params"]["validation_path"] = "/content/flow-forecast/09405500AZC_flow.csv"
data["dataset_params"]["test_path"] = "/content/flow-forecast/09405500AZC_flow.csv"
data["training_params"]["batch_size"] = 100


In [116]:
df = pd.read_csv("09405500AZC_flow.csv")
df = df.dropna(subset=["hour_updated", "cfs", "p01m", "tmpf", "dwpf"])
df.to_csv("09405500AZC_flow.csv")

In [ ]:
t = train_function("PyTorch", data)

Weights sucessfully loaded
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
Using Wandb config:
{}
Torch is using cpu
Using generic dataset params
running torch_single_train
The running loss is: 
271.74462132155895
The number of items in train is: 300
The loss for epoch 0
0.9058154044051965
Computing validation loss
running torch_single_train
The running loss is: 
211.02890829741955
The number of items in train is: 300
The loss for epoch 1
0.7034296943247318
Computing validation loss
running torch_single_train
The running loss is: 
201.40707471966743
The number of items in train is: 300
The loss for epoch 2
0.6713569157322248
Computing validation loss
running torch_single_train
The running loss is: 
196.50829488039017
The number of items in t

In [109]:
from flood_forecast.deployment.inference import InferenceMode
t.params["dataset_params"]["scaler"] = "StandardScaler"
t.params["inference_params"]["dataset_params"]["scaler"] = "Standard_Scaerr"
t.params["inference_params"]["dataset_params"]["scaling"] = "StandardScaler"

In [110]:
a1 = InferenceMode(336, 30, t.params, "/content/flow-forecast/09405500AZC_flow.csv", "/content/flow-forecast/10_November_202102_15PM_model.pth", "eeqwjipfejiowq3grhiu8y7134rewuiho")
a1.make_plots(datetime(2020, 3, 13))

Weights sucessfully loaded
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now


Torch is using cpu


This model is currently forecasting for: 4 targets
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
CSV Path below
/content/flow-forecast/09405500AZC_flow.csv
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
(4, 336, 1, 30)


/content/flow-forecast/flood_forecast/deployment/inference.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(tensor([[[-0.2552, -0.1520, -0.1077,  0.7087],
          [-0.2457, -0.1349, -0.1228,  0.7081],
          [-0.2599, -0.1387, -0.1405,  0.7179],
          ...,
          [-0.5028, -0.1225, -0.1745,  1.1837],
          [-0.4955, -0.1213, -0.1619,  1.1982],
          [-0.5277, -0.1527, -0.1587,  1.2217]]]),
 10,
 Figure({
     'data': [{'name': 'preds',
               'type': 'scatter',
               'x': array([51679, 51680, 51681, ..., 52022, 52023, 52024]),
               'y': array([ 0.        ,  0.        ,  0.        , ..., 45.95415497, 46.13995743,
                           46.44147873])},
              {'name': 'dwpf',
               'type': 'scatter',
               'x': array([51679, 51680, 51681, ..., 52022, 52023, 52024]),
               'y': array([37.4, 37.4, 42.2, ..., 15.2, 17.6, 16.4])},
              {'fill': 'toself',
               'name': '95% confidence interval',
               'type': 'scatter',
               'x': [51679, 51680, 51681, ..., 51681, 51680, 51679],

In [114]:
a1.make_plots(datetime(2020, 3, 13), wandb_plot_id="3013012")

This model is currently forecasting for: 4 targets
interpolate should be below
[]
Now loading /content/flow-forecast/09405500AZC_flow.csv
scaling now
CSV Path below
/content/flow-forecast/09405500AZC_flow.csv
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
Add debugging crap below
(4, 336, 1, 30)


/content/flow-forecast/flood_forecast/deployment/inference.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Shape of unscaled is: 
torch.Size([1, 336, 4])
plotting with CI now
plotting with CI now
plotting with CI now
plotting with CI now


(tensor([[[-0.2552, -0.1520, -0.1077,  0.7087],
          [-0.2457, -0.1349, -0.1228,  0.7081],
          [-0.2599, -0.1387, -0.1405,  0.7179],
          ...,
          [-0.5028, -0.1225, -0.1745,  1.1837],
          [-0.4955, -0.1213, -0.1619,  1.1982],
          [-0.5277, -0.1527, -0.1587,  1.2217]]]),
 10,
 Figure({
     'data': [{'name': 'preds',
               'type': 'scatter',
               'x': array([51679, 51680, 51681, ..., 52022, 52023, 52024]),
               'y': array([ 0.        ,  0.        ,  0.        , ..., 45.95415497, 46.13995743,
                           46.44147873])},
              {'name': 'dwpf',
               'type': 'scatter',
               'x': array([51679, 51680, 51681, ..., 52022, 52023, 52024]),
               'y': array([37.4, 37.4, 42.2, ..., 15.2, 17.6, 16.4])},
              {'fill': 'toself',
               'name': '95% confidence interval',
               'type': 'scatter',
               'x': [51679, 51680, 51681, ..., 51681, 51680, 51679],